In [16]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from csv import writer

In [36]:
station_latitude = pd.read_csv("station_data.csv",index_col=0)
station_latitude = station_latitude[['device_id','latitude','longitude']]
station_latitude_dic = station_latitude.set_index('device_id').T.to_dict('list')
station_latitude_dic = {tuple(v):k for k,v in station_latitude_dic.items()}

all_latitudes = list(station_latitude_dic.keys())
def find_k_nearest(src, latitudes, k):
    def euclidean_dist(p1,p2):
        return (p1[0]-p2[0])**2+(p1[1]-p2[1])**2
    all_distances = [ (coord[0],coord[1], euclidean_dist(src,coord)) for coord in latitudes]
    return sorted(all_distances, key= lambda x:x[2])[1:k+1] #skips 1st point which is itself


k_nearest_dic = {}

for coord, stn in station_latitude_dic.items():
    k_nearest_dic[stn] = list(map(lambda x: (station_latitude_dic[x[:2]],x[2]), find_k_nearest(coord, all_latitudes, 5) ))
k_nearest_dic


{'S77': [('S223', 0.00013491920000006572),
  ('S222', 0.00013876849999994398),
  ('S120', 0.0002564515999999745),
  ('S226', 0.0004343725000000764),
  ('S79', 0.0006549799999997879)],
 'S109': [('S216', 0.00027998660000008377),
  ('S08', 0.0005280999999997645),
  ('S217', 0.0007122037000000591),
  ('S219', 0.0007543610000001618),
  ('S88', 0.0011366899999999854)],
 'S64': [('S218', 0.000399022600000149),
  ('S114', 0.000923849999999812),
  ('S40', 0.0013436623999999104),
  ('S100', 0.0013491025000001455),
  ('S121', 0.001524009999999613)],
 'S90': [('S120', 0.00010853960000001625),
  ('S213', 0.00011508889999999012),
  ('S111', 0.0003758624999998223),
  ('S46', 0.0005785000000002053),
  ('S223', 0.0006418792000002979)],
 'S61': [('S212', 0.00021874409999983975),
  ('S113', 0.00040060040000012306),
  ('S84', 0.000943780000000015),
  ('S215', 0.001093466599999638),
  ('S78', 0.0012179018000000772)],
 'S114': [('S121', 0.000107847999999967),
  ('S218', 0.0006541305999995478),
  ('S229', 0

In [37]:
rain_data = pd.read_csv("rain_data_full.csv", index_col=0)
rain_data

/Users/duantianyu/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,date,time,station,value
0,2023-02-03,00:05:00,S77,0.0
1,2023-02-03,00:10:00,S77,0.0
2,2023-02-03,00:15:00,S77,0.0
3,2023-02-03,00:20:00,S77,0.0
4,2023-02-03,00:25:00,S77,0.0
...,...,...,...,...
25727377,2020-09-09,23:35:00,S105,NaN
25727378,2020-09-09,23:40:00,S105,NaN
25727379,2020-09-09,23:45:00,S105,NaN
25727380,2020-09-09,23:50:00,S105,NaN


In [38]:
rain_data_pivot = rain_data.pivot(index=["date","time"], columns="station", values="value")
rain_data_pivot

station              S07  S08  S100  S102  S104  S105  S106  S107  S108  S109  \
date       time                                                                 
2020-01-01 00:05:00  0.0  0.0   0.0   NaN   0.0   0.0   0.0   0.0   0.0   0.0   
           00:10:00  0.0  0.0   0.0   NaN   0.0   0.0   0.0   0.0   0.0   0.0   
           00:15:00  0.0  0.0   0.0   NaN   0.0   0.0   0.0   NaN   0.0   0.0   
           00:20:00  0.0  0.0   0.0   NaN   0.0   0.0   0.0   0.0   0.0   0.0   
           00:25:00  0.0  0.0   0.0   NaN   0.0   0.0   0.0   0.0   0.0   0.0   
...                  ...  ...   ...   ...   ...   ...   ...   ...   ...   ...   
2023-02-28 23:35:00  NaN  2.4   0.0   NaN   0.2   NaN   6.2   0.8   1.0   3.4   
           23:40:00  NaN  4.2   0.0   NaN   0.2   NaN   5.4   1.8   1.6   3.8   
           23:45:00  NaN  3.4   0.0   NaN   0.0   NaN   4.8   2.8   3.0   1.6   
           23:50:00  NaN  1.6   0.2   NaN   0.0   NaN   2.4   2.0   3.4   1.8   
           23:55:00  NaN  2.2   0.0   NaN   0.4   NaN   4.0   1.0   3.6   1.8   

station              ...  S81  S82  S84  S88  S89  S90  S900  S91  S92  S94  
date       time      ...                                                     
2020-01-01 00:05:00  ...  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  
           00:10:00  ...  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  
           00:15:00  ...  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  
           00:20:00  ...  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  
           00:25:00  ...  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  
...                  ...  ...  ...  ...  ...  ...  ...   ...  ...  ...  ...  
2023-02-28 23:35:00  ...  0.6  NaN  0.8  3.2  1.0  1.6   2.0  NaN  NaN  0.2  
           23:40:00  ...  4.2  NaN  0.6  4.0  0.4  3.8   2.8  NaN  NaN  0.2  
           23:45:00  ...  4.0  NaN  0.6  1.6  0.2  4.6   NaN  NaN  NaN  0.8  
           23:50:00  ...  3.2  NaN  0.6  2.0  0.6  5.4   0.4  NaN  NaN  2.4  
           23:55:00  ...  2.4  NaN  2.6  2.2  1.4  2.6   0.6  NaN  NaN  4.6  

[317622 rows x 81 columns]

In [39]:
rain_data_pivot = rain_data_pivot.to_dict('index')


In [40]:
def get_time_tuples(date_time_tuple):
    date_str, time_str = date_time_tuple
    datetime_obj = datetime.strptime(date_str + ' ' + time_str, '%Y-%m-%d %H:%M:%S')
    result = []
    for i in range(7,13):
        new_datetime_obj = datetime_obj - timedelta(minutes=i*5)
        new_date_str = new_datetime_obj.strftime('%Y-%m-%d')
        new_time_str = new_datetime_obj.strftime('%H:%M:%S')
        result.append((new_date_str, new_time_str))
    return result


In [51]:
header = ["date", "time", "station", "value"]
for time_indx in range(1,7):
    for stn_indx in range(1,7):
        header.append(f'T{time_indx}S{stn_indx}_time')
        header.append(f'T{time_indx}S{stn_indx}_station number')
        header.append(f'T{time_indx}S{stn_indx}_dist')
        header.append(f'T{time_indx}S{stn_indx}_value')

with open('sliding_window_data.csv',"a", encoding='utf-8', newline='') as file:
    writer_object = writer(file)
    writer_object.writerow(header)
    file.close()

In [52]:
lst_of_rows = []
for i in range(1,len(rain_data)):
    orig_date, orig_time, orig_stn, orig_val = rain_data.iloc[i,0], rain_data.iloc[i,1], rain_data.iloc[i,2], rain_data.iloc[i,3]
    nearest_k_stns = k_nearest_dic[orig_stn]
    newRow = [orig_date, orig_time, orig_stn, orig_val]
    # rowIndex = rain_data.index[i]
    for time_indx,(date,time_stamp) in enumerate(get_time_tuples((orig_date, orig_time)),start=1):
        for (stn,dist) in nearest_k_stns:
            newRow.append(-5*time_indx-30)
            newRow.append(stn)
            newRow.append(dist)
            newRow.append(rain_data_pivot.get((date,time_stamp),{}).get(stn,np.nan))
        newRow.append(-5*time_indx-30)
        newRow.append(orig_stn)
        newRow.append(0)
        newRow.append(rain_data_pivot.get((date,time_stamp),{}).get(orig_stn,np.nan))
    lst_of_rows.append(newRow)
    if i % 10000 == 0:
        print(f"Currently writing up till iteration {i}")
        #write multiple rows
        with open('sliding_window_data.csv',"a", encoding='utf-8', newline='') as file:
            writer_object = writer(file)
            writer_object.writerows(lst_of_rows)
        lst_of_rows = []

#write last batch
with open('sliding_window_data.csv',"a", encoding='utf-8', newline='') as file:
    writer_object = writer(file)
    writer_object.writerows(lst_of_rows)

Currently writing up till iteration 10000
Currently writing up till iteration 20000
Currently writing up till iteration 30000
Currently writing up till iteration 40000
Currently writing up till iteration 50000
Currently writing up till iteration 60000
Currently writing up till iteration 70000
Currently writing up till iteration 80000
Currently writing up till iteration 90000
Currently writing up till iteration 100000
Currently writing up till iteration 110000
Currently writing up till iteration 120000
Currently writing up till iteration 130000
Currently writing up till iteration 140000
Currently writing up till iteration 150000
Currently writing up till iteration 160000
Currently writing up till iteration 170000
Currently writing up till iteration 180000
Currently writing up till iteration 190000
Currently writing up till iteration 200000
Currently writing up till iteration 210000
Currently writing up till iteration 220000
Currently writing up till iteration 230000
Currently writing up

Currently writing up till iteration 1900000
Currently writing up till iteration 1910000
Currently writing up till iteration 1920000
Currently writing up till iteration 1930000
Currently writing up till iteration 1940000
Currently writing up till iteration 1950000
Currently writing up till iteration 1960000
Currently writing up till iteration 1970000
Currently writing up till iteration 1980000
Currently writing up till iteration 1990000
Currently writing up till iteration 2000000
Currently writing up till iteration 2010000
Currently writing up till iteration 2020000
Currently writing up till iteration 2030000
Currently writing up till iteration 2040000
Currently writing up till iteration 2050000
Currently writing up till iteration 2060000
Currently writing up till iteration 2070000
Currently writing up till iteration 2080000
Currently writing up till iteration 2090000
Currently writing up till iteration 2100000
Currently writing up till iteration 2110000
Currently writing up till iterat

Currently writing up till iteration 3770000
Currently writing up till iteration 3780000
Currently writing up till iteration 3790000
Currently writing up till iteration 3800000
Currently writing up till iteration 3810000
Currently writing up till iteration 3820000
Currently writing up till iteration 3830000
Currently writing up till iteration 3840000
Currently writing up till iteration 3850000
Currently writing up till iteration 3860000
Currently writing up till iteration 3870000
Currently writing up till iteration 3880000
Currently writing up till iteration 3890000
Currently writing up till iteration 3900000
Currently writing up till iteration 3910000
Currently writing up till iteration 3920000
Currently writing up till iteration 3930000
Currently writing up till iteration 3940000
Currently writing up till iteration 3950000
Currently writing up till iteration 3960000
Currently writing up till iteration 3970000
Currently writing up till iteration 3980000
Currently writing up till iterat

Currently writing up till iteration 5640000
Currently writing up till iteration 5650000
Currently writing up till iteration 5660000
Currently writing up till iteration 5670000
Currently writing up till iteration 5680000
Currently writing up till iteration 5690000
Currently writing up till iteration 5700000
Currently writing up till iteration 5710000
Currently writing up till iteration 5720000
Currently writing up till iteration 5730000
Currently writing up till iteration 5740000
Currently writing up till iteration 5750000
Currently writing up till iteration 5760000
Currently writing up till iteration 5770000
Currently writing up till iteration 5780000
Currently writing up till iteration 5790000
Currently writing up till iteration 5800000
Currently writing up till iteration 5810000
Currently writing up till iteration 5820000
Currently writing up till iteration 5830000
Currently writing up till iteration 5840000
Currently writing up till iteration 5850000
Currently writing up till iterat

Currently writing up till iteration 7510000
Currently writing up till iteration 7520000
Currently writing up till iteration 7530000
Currently writing up till iteration 7540000
Currently writing up till iteration 7550000
Currently writing up till iteration 7560000
Currently writing up till iteration 7570000
Currently writing up till iteration 7580000
Currently writing up till iteration 7590000
Currently writing up till iteration 7600000
Currently writing up till iteration 7610000
Currently writing up till iteration 7620000
Currently writing up till iteration 7630000
Currently writing up till iteration 7640000
Currently writing up till iteration 7650000
Currently writing up till iteration 7660000
Currently writing up till iteration 7670000
Currently writing up till iteration 7680000
Currently writing up till iteration 7690000
Currently writing up till iteration 7700000
Currently writing up till iteration 7710000
Currently writing up till iteration 7720000
Currently writing up till iterat

...

Currently writing up till iteration 16710000
Currently writing up till iteration 16720000
Currently writing up till iteration 16730000
Currently writing up till iteration 16740000
Currently writing up till iteration 16750000
Currently writing up till iteration 16760000
Currently writing up till iteration 16770000
Currently writing up till iteration 16780000
Currently writing up till iteration 16790000
Currently writing up till iteration 16800000
Currently writing up till iteration 16810000
Currently writing up till iteration 16820000
Currently writing up till iteration 16830000
Currently writing up till iteration 16840000
Currently writing up till iteration 16850000
Currently writing up till iteration 16860000
Currently writing up till iteration 16870000
Currently writing up till iteration 16880000
Currently writing up till iteration 16890000
Currently writing up till iteration 16900000
Currently writing up till iteration 16910000
Currently writing up till iteration 16920000
Currently 

Currently writing up till iteration 18540000
Currently writing up till iteration 18550000
Currently writing up till iteration 18560000
Currently writing up till iteration 18570000
Currently writing up till iteration 18580000
Currently writing up till iteration 18590000
Currently writing up till iteration 18600000
Currently writing up till iteration 18610000
Currently writing up till iteration 18620000
Currently writing up till iteration 18630000
Currently writing up till iteration 18640000
Currently writing up till iteration 18650000
Currently writing up till iteration 18660000
Currently writing up till iteration 18670000
Currently writing up till iteration 18680000
Currently writing up till iteration 18690000
Currently writing up till iteration 18700000
Currently writing up till iteration 18710000
Currently writing up till iteration 18720000
Currently writing up till iteration 18730000
Currently writing up till iteration 18740000
Currently writing up till iteration 18750000
Currently 

Currently writing up till iteration 20370000
Currently writing up till iteration 20380000
Currently writing up till iteration 20390000
Currently writing up till iteration 20400000
Currently writing up till iteration 20410000
Currently writing up till iteration 20420000
Currently writing up till iteration 20430000
Currently writing up till iteration 20440000
Currently writing up till iteration 20450000
Currently writing up till iteration 20460000
Currently writing up till iteration 20470000
Currently writing up till iteration 20480000
Currently writing up till iteration 20490000
Currently writing up till iteration 20500000
Currently writing up till iteration 20510000
Currently writing up till iteration 20520000
Currently writing up till iteration 20530000
Currently writing up till iteration 20540000
Currently writing up till iteration 20550000
Currently writing up till iteration 20560000
Currently writing up till iteration 20570000
Currently writing up till iteration 20580000
Currently 

Currently writing up till iteration 22200000
Currently writing up till iteration 22210000
Currently writing up till iteration 22220000
Currently writing up till iteration 22230000
Currently writing up till iteration 22240000
Currently writing up till iteration 22250000
Currently writing up till iteration 22260000
Currently writing up till iteration 22270000
Currently writing up till iteration 22280000
Currently writing up till iteration 22290000
Currently writing up till iteration 22300000
Currently writing up till iteration 22310000
Currently writing up till iteration 22320000
Currently writing up till iteration 22330000
Currently writing up till iteration 22340000
Currently writing up till iteration 22350000
Currently writing up till iteration 22360000
Currently writing up till iteration 22370000
Currently writing up till iteration 22380000
Currently writing up till iteration 22390000
Currently writing up till iteration 22400000
Currently writing up till iteration 22410000
Currently 

Currently writing up till iteration 24030000
Currently writing up till iteration 24040000
Currently writing up till iteration 24050000
Currently writing up till iteration 24060000
Currently writing up till iteration 24070000
Currently writing up till iteration 24080000
Currently writing up till iteration 24090000
Currently writing up till iteration 24100000
Currently writing up till iteration 24110000
Currently writing up till iteration 24120000
Currently writing up till iteration 24130000
Currently writing up till iteration 24140000
Currently writing up till iteration 24150000
Currently writing up till iteration 24160000
Currently writing up till iteration 24170000
Currently writing up till iteration 24180000
Currently writing up till iteration 24190000
Currently writing up till iteration 24200000
Currently writing up till iteration 24210000
Currently writing up till iteration 24220000
Currently writing up till iteration 24230000
Currently writing up till iteration 24240000
Currently 

In [ ]:
rain_data.iloc[25727381:,:]

,date,time,station,value
25727381,2020-09-09,23:55:00,S105,NaN


In [26]:
with open('sliding_window_data.csv',"a", encoding='utf-8', newline='') as file:
    writer_object = writer(file)
    writer_object.writerows(lst_of_rows)

In [ ]:
len(rain_data)

25727382

In [50]:
get_time_tuples(('2023-02-03', '06:30:00'))

[('2023-02-03', '05:55:00'),
 ('2023-02-03', '05:50:00'),
 ('2023-02-03', '05:45:00'),
 ('2023-02-03', '05:40:00'),
 ('2023-02-03', '05:35:00'),
 ('2023-02-03', '05:30:00')]

In [49]:
for time_indx,(date,time_stamp) in enumerate(get_time_tuples(('2023-02-03', '06:30:00')),start=1):
    print(-5*time_indx-30)

-35
-40
-45
-50
-55
-60


In [9]:
k_nearest_dic['S77']

[('S223', 0.00013491920000006572),
 ('S222', 0.00013876849999994398),
 ('S120', 0.0002564515999999745),
 ('S226', 0.0004343725000000764),
 ('S79', 0.0006549799999997879)]